In [4]:
from konlpy.tag import Okt

In [5]:
import re

한글과 공백을 제외한 모든 문자를 제거

In [6]:
text='65가갸어어124,asdf,d~!'

In [7]:
review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',text)

In [8]:
review_text

'가갸어어'

In [9]:
okt = Okt()

In [10]:
print(okt.morphs(u'단독입찰보다 복수입찰의 경우'))

['단독', '입찰', '보다', '복수', '입찰', '의', '경우']


In [11]:
print(okt.nouns(u'유일하게 항공기 체계 종합개발 경험을 갖고 있는 KAI는'))

['항공기', '체계', '종합', '개발', '경험']


In [12]:
print(okt.phrases(u'날카로운 분석과 신뢰감 있는 진행으로'))

['날카로운 분석', '날카로운 분석과 신뢰감', '날카로운 분석과 신뢰감 있는 진행', '분석', '신뢰', '진행']


In [13]:
print(okt.pos(u'이것도 되나욬ㅋㅋ'))

[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나욬', 'Noun'), ('ㅋㅋ', 'KoreanParticle')]


In [14]:
print(okt.pos(u'이것도 되나욬ㅋㅋ', norm=True))

[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나요', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]


In [15]:
print(okt.pos(u'이것도 되나욬ㅋㅋ', norm=True, stem=True))

[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되다', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]


https://konlpy.org/ko/v0.6.0/api/konlpy.tag/

'ㅋㅋㅋㅋㅋ', 'ㅋㅋㅋㅋ', 'ㅋㅋ', 같은 같은 것이 반복되는 것들을 통일시키기

In [16]:
from soynlp.normalizer import emoticon_normalize

In [17]:
print(emoticon_normalize('ㅋㅋㅋㅋㅋㅋ', num_repeats=2))

ㅋㅋ


In [18]:
print(emoticon_normalize('ㅋㅋㅋㅋ', num_repeats=2))

ㅋㅋ


In [57]:
import csv
import json
from pathlib import Path

testPath = Path("./test.csv")
tidyReviews = [["ID", "Star", "Sum.Polarity", "Avg.Polarity", "Num.Verb", "Num.Adjective", "Num.Noun"]]
rootWordDict = list()
cache = list()

def sentiment(word) -> int:
    global cache
    jsonPath = Path("./SentiWord_info.json")
    with jsonPath.open(encoding = "UTF-8") as f:
        rootWordDict = json.load(f)
    result = 0
    for wordcell in cache:
        if word == wordcell["word"]:
            result = int(wordcell["polarity"])
            return result
    
    for wordcell in rootWordDict:
        if word == wordcell["word"]:
            result = int(wordcell["polarity"])
            if len(cache) < 50: cache.append(wordcell)
            else: 
                cache.pop(0)
                cache.append(wordcell)
            break
    return result

def average(top, bottom):
    if bottom == 0: return "N/A"
    return round(top/bottom, 2)


with testPath.open(encoding = "UTF-8") as f:
    reader = csv.reader(f)
    
    for i, row in enumerate(reader):
        if i == 0: continue
        buffer = [row[0]]
        score = int(re.sub("[^0-9]", '', row[1]))
        polarity = 0
        verb = 0
        adjective = 0
        noun = 0
        buffer.append(score)
        singleReview = okt.pos(emoticon_normalize(row[2], num_repeats=1), norm = True, stem = True)
        lengthOfSingleReview = len(singleReview)
        for word, part in singleReview:
            if part == "Noun": noun += 1
            elif part == "Verb": verb += 1
            elif part == "Adjective": adjective += 1
            else: continue
            if sentiment(word) == 0: 
                if part == "Noun": noun -= 1
                elif part == "Verb": verb -= 1
                elif part == "Adjective": adjective -= 1
                continue
            polarity += sentiment(word)
        if verb+adjective+noun == 0: continue
        buffer += [polarity, average(polarity, verb+adjective+noun), verb, adjective, noun]
        tidyReviews.append(buffer)


In [56]:
with open("review.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(tidyReviews)